In [17]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (99.8 kB/s)
Reading package li

In [18]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-02 21:04:46--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2022-10-02 21:04:48 (1.18 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [20]:
from pyspark import SparkFiles
url = "https://amazon-officereviews.s3.amazonaws.com/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.|2015-08-31 00:00:00|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| 

### Create DataFrames to match tables

In [21]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [22]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43622307|             1|
|   33972406|             1|
|     108460|             1|
|   43515569|             1|
|     132406|             1|
|   42560427|             1|
|    1673863|             2|
|   14552054|             1|
|   45632184|             1|
|   14703850|             2|
+-----------+--------------+
only showing top 10 rows



In [23]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates(["product_id"])

products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0007262973|Pastels (Collins ...|
|0113000316|123GetInk -14-pac...|
|0136039847|Pearson MyHistory...|
|0307382214|Happy Baby, Healt...|
|0395742889|Webster's II New ...|
|0439165210|Alpha Tales Learn...|
|043928631X|Harry Potter Lent...|
|0439394058|Scholastic SC9394...|
|0439492092|Scholastic Teache...|
|043949219X|Scholastic ¡Color...|
+----------+--------------------+
only showing top 10 rows



In [9]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R18RVCKGH1SSI9|   43081963|B001BM2MAC|     307809868| 2015-08-31|
|R3L4L6LW1PUOFY|   10951564|B00DZYEXPQ|      75004341| 2015-08-31|
|R2J8AWXWTDX2TF|   21143145|B00RTMUHDW|     529689027| 2015-08-31|
|R1PR37BR7G3M6A|   52782374|B00D7H8XB6|     868449945| 2015-08-31|
|R3BDDDZMZBZDPU|   24045652|B001XCWP34|      33521401| 2015-08-31|
| R8T6MO75ND212|   21751234|B004J2NBCO|     214932869| 2015-08-31|
|R2YWMQT2V11XYZ|    9109358|B00MOPAG8K|     863351797| 2015-08-31|
|R1V2HYL6OI9V39|    9967215|B003AHIK7U|     383470576| 2015-08-31|
|R3BLQBKUNXGFS4|   11234247|B006TKH2RO|     999128878| 2015-08-31|
|R17MOWJCAR9Y8Q|   12731488|B00W61M9K0|     622066861| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [41]:
# We need to convert the Star Rating Column into an Interger to be able to write the vine_table in RDS
# since the table in PgAdmin had star_rating as an in
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

vine1_df = df.withColumn("star_rating",col("star_rating").cast("Integer"))
vine1_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'timestamp')]

In [42]:
vine1_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.|2015-08-31 00:00:00|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| 

In [43]:
# Create the vine_table DataFrame
vine_df = vine1_df.select(["review_id", "star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18RVCKGH1SSI9|          5|            0|          0|   N|                Y|
|R3L4L6LW1PUOFY|          5|            0|          1|   N|                Y|
|R2J8AWXWTDX2TF|          5|            0|          0|   N|                Y|
|R1PR37BR7G3M6A|          1|            2|          3|   N|                Y|
|R3BDDDZMZBZDPU|          4|            0|          0|   N|                Y|
| R8T6MO75ND212|          5|            0|          0|   N|                Y|
|R2YWMQT2V11XYZ|          5|            0|          0|   N|                N|
|R1V2HYL6OI9V39|          5|            6|          6|   N|                Y|
|R3BLQBKUNXGFS4|          5|            0|          0|   N|                Y|
|R17MOWJCAR9Y8Q|          5|            0|          0|   N|     

In [44]:
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [45]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb16.crre5eooasmv.us-east-1.rds.amazonaws.com:5432/Mod16challenge"
config = {"user":"root", 
          "password": "root2022", 
          "driver":"org.postgresql.Driver"}

In [12]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [13]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [14]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [46]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)